In [1]:
import networkx as nx
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
allgenes = pd.read_csv('data/all_genes.txt', sep='\n', header=[0])
mappings = df = pd.DataFrame(pd.read_excel("data/mapping.xlsx"))
# print(mappings.shape)
# print(mappings.head())
mappings=pd.merge(allgenes, mappings, on="Entrenz")
# print(mappings.head())
# print(mappings.shape)
p_list = mappings['Symbol'].tolist()
# protein_list=['TPH1','COMT','SLC18A2','HTR1B','HTR2C','HTR2A','MAOA',
#             'TPH2','HTR1A','HTR7','SLC6A4','GABBR2','POMC','GNAI3',
#             'NPY','ADCY1','PDYN','GRM2','GRM3','GABBR1']

In [4]:
lines=[]
for i in range(0,len(p_list),20):
    protein_list=p_list[i:i+20]
    proteins = '%0d'.join(protein_list)
    # print(protein_list)
    url = 'https://string-db.org/api/tsv/network?identifiers=' + proteins + '&species=9606'
    r = requests.get(url)
    if(r.status_code==200):
        lines.extend(r.text.split('\n')) # pull the text from the response object and split based on new lines
# print(lines)
data = [l.split('\t') for l in lines] # split each line into its components based on tabs
# convert to dataframe using the first row as the column names; drop empty, final row
df = pd.DataFrame(data[1:-1], columns = data[0]) 
# dataframe with the preferred names of the two proteins and the score of the interaction
interactions = df[['preferredName_A', 'preferredName_B', 'score']]


['stringId_A\tstringId_B\tpreferredName_A\tpreferredName_B\tncbiTaxonId\tscore\tnscore\tfscore\tpscore\tascore\tescore\tdscore\ttscore', '9606.ENSP00000264012\t9606.ENSP00000484928\tCDH3\tCDH4\t9606\t0.569\t0\t0\t0\t0\t0\t0.54\t0.649', '9606.ENSP00000264012\t9606.ENSP00000484928\tCDH3\tCDH4\t9606\t0.569\t0\t0\t0\t0\t0\t0.54\t0.649', '9606.ENSP00000264012\t9606.ENSP00000269141\tCDH3\tCDH2\t9606\t0.575\t0\t0\t0\t0\t0\t0.54\t0.812', '9606.ENSP00000264012\t9606.ENSP00000269141\tCDH3\tCDH2\t9606\t0.575\t0\t0\t0\t0\t0\t0.54\t0.812', '9606.ENSP00000264933\t9606.ENSP00000411825\tPDCD6\tPDCD6IP\t9606\t0.998\t0\t0\t0\t0.081\t0.898\t0\t0.981', '9606.ENSP00000264933\t9606.ENSP00000411825\tPDCD6\tPDCD6IP\t9606\t0.998\t0\t0\t0\t0.081\t0.898\t0\t0.981', '9606.ENSP00000269141\t9606.ENSP00000484928\tCDH2\tCDH4\t9606\t0.67\t0\t0\t0\t0.076\t0.27\t0.54\t0.799', '9606.ENSP00000269141\t9606.ENSP00000484928\tCDH2\tCDH4\t9606\t0.67\t0\t0\t0\t0.076\t0.27\t0.54\t0.799', '9606.ENSP00000337513\t9606.ENSP000003653

In [11]:
print(interactions.shape)
igindex=[]
for index, row in interactions.iterrows():
    if('Gene' not in row['preferredName_A'] or ('Gene' not in row['preferredName_B'])):
        igindex.append(index)
interactions = interactions.drop(labels=igindex, axis=0)
print(interactions.shape)
print("before cosine calc, the scores from STRING\n",interactions.head())
interactions.to_csv('string_interactions.csv', encoding='utf-8', index=False)

(3776, 3)
(3776, 3)
before cosine calc, the scores from STRING
   preferredName_A preferredName_B  score
0       Gene_1001       Gene_1002  0.569
1       Gene_1001       Gene_1002  0.569
2       Gene_1001       Gene_1000  0.575
3       Gene_1001       Gene_1000  0.575
4      Gene_10016      Gene_10015  0.998


In [7]:
interactions=interactions.dropna()
print(interactions.head())
print(interactions.shape)
# mappings.columns = mappings.iloc[0] 
print(mappings.shape)
for ind1, row1 in mappings.iterrows():
    for index, row in interactions.iterrows():
        if(row['preferredName_A'].lower()==row1['Symbol'].lower()):
            row['preferredName_A']=row1['Entrenz']
        if(row['preferredName_B'].lower()==row1['Symbol'].lower()):
            row['preferredName_B']=row1['Entrenz']
    print(ind1,end="")




  preferredName_A preferredName_B  score
0       Gene_1001       Gene_1002  0.569
1       Gene_1001       Gene_1002  0.569
2       Gene_1001       Gene_1000  0.575
3       Gene_1001       Gene_1000  0.575
4      Gene_10016      Gene_10015  0.998
(6199, 3)
(5186, 2)
01234567891011121314151617181920212223242526272829303132333435363738394041424344454647484950515253545556575859606162636465666768697071727374757677787980818283848586878889909192939495969798991001011021031041051061071081091101111121131141151161171181191201211221231241251261271281291301311321331341351361371381391401411421431441451461471481491501511521531541551561571581591601611621631641651661671681691701711721731741751761771781791801811821831841851861871881891901911921931941951961971981992002012022032042052062072082092102112122132142152162172182192202212222232242252262272282292302312322332342352362372382392402412422432442452462472482492502512522532542552562572582592602612622632642652662672682692702712722732742752762772782792802

KeyboardInterrupt: 

In [ ]:
import multiprocessing
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import os
import sys
import json
print(interactions.shape)
print("before cosine calc, the scores from STRING",interactions.head())
def load_embedding(path):
    embedding = KeyedVectors.load_word2vec_format(path, binary=True)
    print('embedding loaded from', path)
    return embedding
w2v_model = load_embedding("bioconceptvec_word2vec_cbow.bin")

def cosine(a, b):
    norm1 = np.linalg.norm(a)
    norm2 = np.linalg.norm(b)
    return np.dot(a, b) / (norm1 * norm2)
ignoreindex=[]
prediction=[]
for index, row in interactions.iterrows():
    if('Gene' not in row['preferredName_A'] or ('Gene' not in row['preferredName_B'])):
        ignoreindex.append(index)
    else:
        row['score']=cosine(w2v_model[row['preferredName_A']], w2v_model[row['preferredName_B']])
trueval=0
print("after cosine calc:",interactions.head())
for index, row in interactions.iterrows():
    if(index not in ignoreindex ):
        if(row['score']<0.5):
            prediction.append(0)
        else:
            prediction.append(1)
            trueval+=1
print(trueval) # 90 predicted as true
print(len(ignoreindex)) # didn't have their entrenz
print(len(prediction)) # 90 as true and 64 as false
            



In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
y_true=[1]*len(prediction)
print(precision_recall_fscore_support(y_true, prediction, average='macro'))

target_names = ['0', '1'] # 0 means no interation, 1 means identified as interaction as cosine score>0.5
print(classification_report(y_true, prediction, target_names=target_names))
